In [ ]:
# | hide
import nbdev
from nbdev.showdoc import *

nbdev.nbdev_export()

### Functional operations


In [ ]:
# | default_exp functional

In [ ]:
# | export
from tidygrad.tensor import Tensor, BaseOp, UnaryElementwiseOp, BinaryElementwiseOp
import numpy as np

In [ ]:
# | exporti
class Sigmoid(UnaryElementwiseOp):
    """Take the sigmoid of a tensor"""

    name_template = "sigmoid({})"

    def __init__(self, a, name=None):
        super().__init__(a, name=name)
        self.out = Tensor(1 / (1 + np.exp(-self.args[0].data)), name=self.name, op=self)

    def backward(self):
        self.parents[0].grad += self.out.grad * self.out.data * (1 - self.out.data)

In [ ]:
# | export
def sigmoid(input, name=None):
    return Sigmoid(input, name=name).out

In [ ]:
# | exporti
class Relu(UnaryElementwiseOp):
    """Take the sigmoid of a tensor"""

    name_template = "relu({})"

    def __init__(self, a, name=None):
        super().__init__(a, name=name)
        self.out = Tensor(np.maximum(0, self.args[0].data), name=self.name, op=self)

    def backward(self):
        self.parents[0].grad += self.out.grad * (self.out.data > 0)

In [ ]:
# | export
def relu(input, name=None):
    return Relu(input, name=name).out

In [ ]:
#| export
from tidygrad.tensor import ExpLog

In [ ]:
#| export

def BCE_loss(logits, target):
    return -logits + logits * target - ExpLog(-logits).out
